In [1]:
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [25]:
m = AbstractModel()

m.GRADES = Set()
m.TIMESLOTS = Set()
m.ROOMS = Set()

m.pop = Param(m.GRADES)
m.cap = Param(m.ROOMS)

m.Assign = Var(m.GRADES, m.TIMESLOTS, m.ROOMS, domain = Binary)

def one_grade_in_one_timeslot_and_one_room(model, t, r):
    return quicksum(model.Assign[g, t, r] for g in model.GRADES) <= 1

def one_room_with_one_grade(model, g, t):
    return quicksum(model.Assign[g, t, r] for r in model.ROOMS) <= 1

def capacity_of_room(model, g, t, r):
    return model.Assign[g, t, r] * model.pop[g] <= model.cap[r]

m.One_Timeslot_per_grade = Constraint(m.TIMESLOTS, m.ROOMS, rule = one_grade_in_one_timeslot_and_one_room)
m.One_Room_with_one_grade = Constraint(m.GRADES, m.TIMESLOTS, rule = one_room_with_one_grade)
m.capacity_of_room = Constraint(m.GRADES, m.TIMESLOTS, m.ROOMS, rule = capacity_of_room)

m.assign_sum = Objective(rule = lambda model: sum_product(model.Assign), sense = maximize)


def equality(model, g, g_prime):
    if g == g_prime:
        return Constraint.Skip
    return quicksum(model.Assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) == quicksum(model.Assign[g_prime, t, r] for t in model.TIMESLOTS for r in model.ROOMS)


m.equality_rule = Constraint(m.GRADES, m.GRADES, rule = equality)

In [35]:
instanceData = { None: {
    'GRADES': {None: ['1A', '1B', '2A', '2B']},
    'TIMESLOTS': {None: range(1, 9)},
    'ROOMS': {None: ['101', '102', '103']},
    'pop': {'1A': 31, '1B': 36, '2A': 39, '2B': 25},
    'cap': {'101': 40, '102': 50, '103': 35 }
}}

In [36]:
instance = m.create_instance(instanceData)
opt = SolverFactory('glpk')
opt.solve(instance)

{'Problem': [{'Name': 'unknown', 'Lower bound': 24.0, 'Upper bound': 24.0, 'Number of objectives': 1, 'Number of constraints': 164, 'Number of variables': 96, 'Number of nonzeros': 864, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.007402658462524414}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [37]:
face_to_face_time_third_B = 0
for t in instance.TIMESLOTS:
    for r in instance.ROOMS:
        face_to_face_time_third_B += value(instance.Assign['2A', t, r])

In [38]:
31 + 36 + 39 + 25

131

In [39]:
131 * 6

786

In [40]:
max_hours = face_to_face_time_third_B * 131

In [41]:
max_hours

786.0

In [42]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in instance.GRADES:
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            if value(instance.Assign[g, t, r]):
                d[r][t] = g

In [43]:
pd.DataFrame(d).fillna('N/A').sort_index()

,101,102,103
1,1A,1B,2B
2,1B,1A,2B
3,2A,1A,2B
4,2A,1B,1A
5,2A,2B,1A
6,2A,1B,2B
7,2A,1B,1A
8,2A,1B,2B


In [19]:
instanceData = { None: {
    'GRADES': {None: ['1A', '1B', '2A', '2B']},
    'TIMESLOTS': {None: range(8, 16)},
    'ROOMS': {None: ['101', '102', '103']},
    'pop': {'1A': 31, '1B': 36, '2A': 39, '2B': 55},
    'cap': {'101': 40, '102': 50, '103': 35 }
}}

In [20]:
instance = m.create_instance(instanceData)
opt = SolverFactory('glpk')
opt.solve(instance)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 164, 'Number of variables': 96, 'Number of nonzeros': 864, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.00642704963684082}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [23]:
from collections import defaultdict
d = defaultdict(lambda: defaultdict(str))
for g in instance.GRADES:
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            if value(instance.Assign[g, t, r]):
                d[r][t] = g

In [24]:
pd.DataFrame(d).fillna('N/A').sort_index()

""
